In [2]:
import os 
import json
import pandas as pd
import traceback

In [3]:
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
key = os.getenv("GROQ_API_KEY")

In [6]:
from langchain_groq import ChatGroq
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
import PyPDF2


In [8]:
RESPONSE_JSON = {
  "quiz_info": {
    "title": "Quiz Title",
    "subject": "Subject Name",
    "difficulty": "easy/medium/hard",
    "total_questions": 3
  },
  "questions": [
    {
      "id": 1,
      "question": "What is the main topic discussed in the text?",
      "options": {
        "A": "Option A text",
        "B": "Option B text",
        "C": "Option C text",
        "D": "Option D text"
      },
      "correct_answer": "A",
      "explanation": "Brief explanation of why A is correct"
    },
    {
      "id": 2,
      "question": "Which of the following statements is true?",
      "options": {
        "A": "Option A text",
        "B": "Option B text",
        "C": "Option C text",
        "D": "Option D text"
      },
      "correct_answer": "B",
      "explanation": "Brief explanation of why B is correct"
    },
    {
      "id": 3,
      "question": "What would be the best approach for this scenario?",
      "options": {
        "A": "Option A text",
        "B": "Option B text",
        "C": "Option C text",
        "D": "Option D text"
      },
      "correct_answer": "C",
      "explanation": "Brief explanation of why C is correct"
    }
  ]
}

In [9]:
TEMPLATE = """
# MCQ Generation Instructions

## Input Text
{text}

## Task Description
You are an expert MCQ maker. Given the above text, it is your job to create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.

## Requirements
1. **Question Count**: Generate exactly {number} MCQs
2. **Uniqueness**: Ensure no questions are repeated
3. **Accuracy**: All questions must conform to the provided text content
4. **Format**: Follow the RESPONSE_JSON structure below exactly

## Response Format
Please format your response using the following JSON structure as a guide:

### RESPONSE_JSON
{response_json}

## Instructions
- Create clear, well-structured multiple choice questions
- Ensure all options are plausible and relevant
- Provide accurate explanations for correct answers
- Maintain consistency with the specified tone and subject level
"""


In [10]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [11]:
llm = ChatGroq(
    model="llama3-8b-8192",
    groq_api_key=key,
    temperature=0.3,  # Lower temperature for consistent, factual responses
    max_tokens=2048,  # Higher limit for detailed MCQ explanations
    top_p=0.8,       # Slightly lower for more focused responses
    stop=["\n\nQuestion:", "\n\n---", "\n\n###"]  # Stop at question boundaries
)



d:\MCQ_Generator\env\lib\site-packages\langchain_groq\chat_models.py:359: UserWarning: WARNING! top_p is not default parameter.
                    top_p was transferred to model_kwargs.
                    Please confirm that top_p is what you intended.
  warnings.warn(


In [12]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

C:\Users\Dell\AppData\Local\Temp\ipykernel_17712\2669661367.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)


In [13]:
TEMPLATE2 = """
# Quiz Review and Enhancement Instructions

## Input Quiz
{quiz}

## Task Description
You are an expert English grammarian and writer. Given a Multiple Choice Quiz for {subject} students, you need to evaluate the complexity of the questions and provide a complete analysis of the quiz.

## Review Requirements
1. **Complexity Analysis**: Evaluate if questions match student cognitive abilities (max 50 words)
2. **Quality Assessment**: Check grammar, clarity, and educational value
3. **Difficulty Adjustment**: Update questions that don't match student level
4. **Tone Optimization**: Adjust tone to perfectly fit student abilities
5. **Content Validation**: Ensure questions are appropriate for {subject} students

## Analysis Guidelines
- **Cognitive Level**: Assess if questions are too easy, too hard, or just right
- **Language Complexity**: Check vocabulary and sentence structure
- **Question Clarity**: Ensure questions are unambiguous and well-written
- **Option Quality**: Verify distractors are plausible but clearly incorrect
- **Educational Value**: Confirm questions promote learning and understanding

## Enhancement Instructions
- If questions are too complex: Simplify language and reduce difficulty
- If questions are too simple: Increase complexity and add analytical elements
- If tone is inappropriate: Adjust to match student age and subject level
- If grammar is poor: Correct all grammatical and punctuation errors
- If explanations are unclear: Improve clarity and educational value

## Output Format
Provide your analysis and any necessary updates in the following structure:

### COMPLEXITY ANALYSIS
[Your 50-word complexity assessment]

### QUALITY ASSESSMENT
[Grammar, clarity, and educational value review]

### ENHANCED QUIZ
[Updated quiz with improved questions, tone, and difficulty level]

### CHANGES MADE
[List of specific improvements and modifications]

## Instructions
- Be thorough but concise in your analysis
- Focus on educational appropriateness and student engagement
- Maintain the original quiz structure while improving quality
- Ensure all changes enhance learning outcomes
"""

In [14]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [15]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [16]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [17]:
file_path = "D:\MCQ_Generator\data.txt"

In [18]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [19]:
print(TEXT)

Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] "Understanding" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. This image understanding can be seen as the disentangling of symbolic information from image data using models constructed with the aid of geometry, physics, statistics, and learning theory.

The scientific discipline of computer vision is concerned with the theory behind artificial systems that extract information from images. Image data can take many forms, such as video sequences, views from multiple cameras, multi-dimensional data from a 3D scanner, 3D point clouds from LiDaR sensors, or medical scanning devices. Th

In [20]:
# Serialize the Python dictionary into a JSON-formatted string
json.dumps(RESPONSE_JSON)

'{"quiz_info": {"title": "Quiz Title", "subject": "Subject Name", "difficulty": "easy/medium/hard", "total_questions": 3}, "questions": [{"id": 1, "question": "What is the main topic discussed in the text?", "options": {"A": "Option A text", "B": "Option B text", "C": "Option C text", "D": "Option D text"}, "correct_answer": "A", "explanation": "Brief explanation of why A is correct"}, {"id": 2, "question": "Which of the following statements is true?", "options": {"A": "Option A text", "B": "Option B text", "C": "Option C text", "D": "Option D text"}, "correct_answer": "B", "explanation": "Brief explanation of why B is correct"}, {"id": 3, "question": "What would be the best approach for this scenario?", "options": {"A": "Option A text", "B": "Option B text", "C": "Option C text", "D": "Option D text"}, "correct_answer": "C", "explanation": "Brief explanation of why C is correct"}]}'

In [21]:
NUMBER=5 
SUBJECT="computer vision"
TONE="simple"

In [22]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )

C:\Users\Dell\AppData\Local\Temp\ipykernel_17712\1882050128.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use invoke instead.
  response=generate_evaluate_chain(
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")
Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")


Prompt after formatting:

# MCQ Generation Instructions

## Input Text
Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] "Understanding" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. This image understanding can be seen as the disentangling of symbolic information from image data using models constructed with the aid of geometry, physics, statistics, and learning theory.

The scientific discipline of computer vision is concerned with the theory behind artificial systems that extract information from images. Image data can take many forms, such as video sequences, views from multiple cameras, multi-dimensional data from a 3D scann

Error in StdOutCallbackHandler.on_chain_start callback: AttributeError("'NoneType' object has no attribute 'get'")



> Finished chain.
Prompt after formatting:

# Quiz Review and Enhancement Instructions

## Input Quiz
Here are the 5 MCQs in the RESPONSE_JSON format:

```
{
  "quiz_info": {
    "title": "Computer Vision Quiz",
    "subject": "Computer Vision",
    "difficulty": "easy",
    "total_questions": 5
  },
  "questions": [
    {
      "id": 1,
      "question": "What is the main goal of computer vision?",
      "options": {
        "A": "To create artificial intelligence",
        "B": "To extract information from images",
        "C": "To analyze video sequences",
        "D": "To detect objects in images"
      },
      "correct_answer": "B",
      "explanation": "Computer vision aims to extract information from images, which can be used to understand the world and make decisions."
    },
    {
      "id": 2,
      "question": "What is an example of image data?",
      "options": {
        "A": "Video sequences",
        "B": "3D point clouds from LiDaR sensors",
        "C": "Multi-dimen

In [28]:
response 

{'text': 'Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] "Understanding" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. This image understanding can be seen as the disentangling of symbolic information from image data using models constructed with the aid of geometry, physics, statistics, and learning theory.\n\nThe scientific discipline of computer vision is concerned with the theory behind artificial systems that extract information from images. Image data can take many forms, such as video sequences, views from multiple cameras, multi-dimensional data from a 3D scanner, 3D point clouds from LiDaR sensors, or medical scanning

In [30]:
import pandas as pd
from datetime import datetime

In [32]:
# Check what variables you have
print("Available variables:")
print([var for var in dir() if not var.startswith('_')])

# If you have a response, check its name
# Common names might be: response, result, mcqs, mcq_response, etc.

Available variables:
['ChatGroq', 'In', 'LLMChain', 'NUMBER', 'Out', 'PromptTemplate', 'PyPDF2', 'RESPONSE_JSON', 'SUBJECT', 'SequentialChain', 'TEMPLATE', 'TEMPLATE2', 'TEXT', 'TONE', 'datetime', 'display_mcqs_nicely', 'display_parsed_mcqs', 'exit', 'file', 'file_path', 'generate_evaluate_chain', 'get_ipython', 'json', 'key', 'llm', 'load_dotenv', 'open', 'os', 'pd', 'quit', 'quiz', 'quiz_chain', 'quiz_evaluation_prompt', 'quiz_generation_prompt', 'response', 'review_chain', 'save_mcqs_to_csv', 'save_mcqs_to_json', 'traceback']


In [36]:
# Check the type and structure of your response
print(f"Response type: {type(response)}")
print(f"Response content: {response}")

# If it's a dictionary, check its keys
if isinstance(response, dict):
    print(f"Dictionary keys: {list(response.keys())}")
    # Common keys might be: 'content', 'text', 'response', 'result', etc.

Response type: <class 'dict'>
Response content: {'text': 'Computer vision tasks include methods for acquiring, processing, analyzing, and understanding digital images, and extraction of high-dimensional data from the real world in order to produce numerical or symbolic information, e.g. in the form of decisions.[1][2][3][4] "Understanding" in this context signifies the transformation of visual images (the input to the retina) into descriptions of the world that make sense to thought processes and can elicit appropriate action. This image understanding can be seen as the disentangling of symbolic information from image data using models constructed with the aid of geometry, physics, statistics, and learning theory.\n\nThe scientific discipline of computer vision is concerned with the theory behind artificial systems that extract information from images. Image data can take many forms, such as video sequences, views from multiple cameras, multi-dimensional data from a 3D scanner, 3D poin

In [37]:
# Extract the quiz content from your response
quiz_content = response['quiz']
print("🔍 YOUR MCQ QUIZ:")
print("=" * 80)
print(quiz_content)
print("=" * 80)

🔍 YOUR MCQ QUIZ:
Here are the 5 MCQs in the required format:

```
{
  "quiz_info": {
    "title": "Computer Vision Quiz",
    "subject": "Computer Vision",
    "difficulty": "medium",
    "total_questions": 5
  },
  "questions": [
    {
      "id": 1,
      "question": "What is the primary goal of computer vision?",
      "options": {
        "A": "To create artificial intelligence systems",
        "B": "To extract information from images",
        "C": "To analyze and understand digital images",
        "D": "To develop computer graphics"
      },
      "correct_answer": "C",
      "explanation": "Computer vision aims to understand and extract information from digital images, which involves analyzing and processing visual data."
    },
    {
      "id": 2,
      "question": "What type of data can be used in computer vision?",
      "options": {
        "A": "Only 2D images",
        "B": "Only 3D point clouds",
        "C": "Multi-dimensional data from various sources",
        "D": 

In [41]:
import json
import re

# Function to clean and parse JSON quiz format
def parse_json_quiz(quiz_text):
    """Parse JSON quiz format and extract MCQs with error handling"""
    try:
        # Find the JSON part (between ```json and ```)
        if '```json' in quiz_text:
            start = quiz_text.find('```json') + 7
            end = quiz_text.find('```', start)
            json_str = quiz_text[start:end].strip()
        elif '```' in quiz_text:
            start = quiz_text.find('```') + 3
            end = quiz_text.find('```', start)
            json_str = quiz_text[start:end].strip()
        else:
            json_str = quiz_text
        
        # Clean up the JSON string
        # Remove any trailing commas and fix common issues
        json_str = re.sub(r',(\s*[}\]])', r'\1', json_str)  # Remove trailing commas
        json_str = re.sub(r'(\d+)"', r'\1', json_str)  # Fix quotes around numbers
        
        # Parse JSON
        quiz_data = json.loads(json_str)
        return quiz_data
    except Exception as e:
        print(f"Error parsing JSON: {e}")
        print("Attempting to extract questions manually...")
        return extract_questions_manually(quiz_text)

# Function to extract questions manually if JSON parsing fails
def extract_questions_manually(quiz_text):
    """Extract questions manually from the quiz text"""
    questions = []
    lines = quiz_text.split('\n')
    
    current_question = {}
    in_question_block = False
    
    for line in lines:
        line = line.strip()
        
        # Look for question patterns
        if '"question":' in line:
            question_text = line.split('"question":')[1].strip().strip('",')
            current_question = {'question': question_text, 'options': {}, 'id': len(questions) + 1}
            in_question_block = True
            
        elif '"options":' in line:
            in_question_block = True
            
        elif '"A":' in line and in_question_block:
            option_a = line.split('"A":')[1].strip().strip('",')
            current_question['options']['A'] = option_a
            
        elif '"B":' in line and in_question_block:
            option_b = line.split('"B":')[1].strip().strip('",')
            current_question['options']['B'] = option_b
            
        elif '"C":' in line and in_question_block:
            option_c = line.split('"C":')[1].strip().strip('",')
            current_question['options']['C'] = option_c
            
        elif '"D":' in line and in_question_block:
            option_d = line.split('"D":')[1].strip().strip('",')
            current_question['options']['D'] = option_d
            
        elif '"correct_answer":' in line and in_question_block:
            correct_answer = line.split('"correct_answer":')[1].strip().strip('",')
            current_question['correct_answer'] = correct_answer
            
        elif '"explanation":' in line and in_question_block:
            explanation = line.split('"explanation":')[1].strip().strip('",')
            current_question['explanation'] = explanation
            
        elif line == '},' or line == '}' and in_question_block:
            if current_question:
                questions.append(current_question)
                current_question = {}
                in_question_block = False
    
    if current_question:
        questions.append(current_question)
    
    return {
        'quiz_info': {
            'title': 'Computer Vision Quiz',
            'subject': 'Computer Vision',
            'difficulty': 'medium',
            'total_questions': len(questions)
        },
        'questions': questions
    }

# Parse and display the quiz
quiz_data = parse_json_quiz(response['quiz'])

if quiz_data:
    print(f"\n📝 QUIZ INFO:")
    print(f"Title: {quiz_data.get('quiz_info', {}).get('title', 'N/A')}")
    print(f"Subject: {quiz_data.get('quiz_info', {}).get('subject', 'N/A')}")
    print(f"Difficulty: {quiz_data.get('quiz_info', {}).get('difficulty', 'N/A')}")
    print(f"Total Questions: {quiz_data.get('quiz_info', {}).get('total_questions', 'N/A')}")
    
    print(f"\n📋 QUESTIONS:")
    print("=" * 80)
    
    for question in quiz_data.get('questions', []):
        print(f"\n MCQ {question.get('id', 'N/A')}:")
        print("-" * 40)
        print(f"Question: {question.get('question', 'N/A')}")
        print()
        
        options = question.get('options', {})
        for letter in ['A', 'B', 'C', 'D']:
            option_text = options.get(letter, 'N/A')
            print(f"{letter}) {option_text}")
        
        print()
        print(f"✅ Correct Answer: {question.get('correct_answer', 'N/A')}")
        print(f"💡 Explanation: {question.get('explanation', 'N/A')}")
        print("-" * 40)
else:
    print("Failed to parse quiz data")


📝 QUIZ INFO:
Title: Computer Vision Quiz
Subject: Computer Vision
Difficulty: medium
Total Questions: 5

📋 QUESTIONS:

 MCQ 1:
----------------------------------------
Question: What is the primary goal of computer vision?

A) To create artificial intelligence systems
B) To extract information from images
C) To analyze and understand digital images
D) To develop computer graphics

✅ Correct Answer: C
💡 Explanation: Computer vision aims to understand and extract information from digital images, which involves analyzing and processing visual data.
----------------------------------------

 MCQ 2:
----------------------------------------
Question: What type of data can be used in computer vision?

A) Only 2D images
B) Only 3D point clouds
C) Multi-dimensional data from various sources
D) Only text data

✅ Correct Answer: C
💡 Explanation: Computer vision can work with various types of data, including 2D images, 3D point clouds, video sequences, and multi-dimensional data from sensors and 

In [42]:
import pandas as pd
from datetime import datetime

# Function to save MCQs to CSV
def save_mcqs_to_csv(quiz_data, filename=None):
    """Save MCQs to CSV format"""
    if not quiz_data or 'questions' not in quiz_data:
        print("No quiz data to save")
        return
    
    # Create rows for CSV
    rows = []
    for question in quiz_data['questions']:
        row = {
            'Question_Number': question.get('id', ''),
            'Question': question.get('question', ''),
            'Option_A': question.get('options', {}).get('A', ''),
            'Option_B': question.get('options', {}).get('B', ''),
            'Option_C': question.get('options', {}).get('C', ''),
            'Option_D': question.get('options', {}).get('D', ''),
            'Correct_Answer': question.get('correct_answer', ''),
            'Explanation': question.get('explanation', '')
        }
        rows.append(row)
    
    # Create DataFrame
    df = pd.DataFrame(rows)
    
    # Generate filename if not provided
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        subject = quiz_data.get('quiz_info', {}).get('subject', 'quiz').replace(' ', '_').lower()
        filename = f"{subject}_mcqs_{timestamp}.csv"
    
    # Save to CSV
    df.to_csv(filename, index=False, encoding='utf-8')
    print(f"✅ MCQs saved to CSV: {filename}")
    print(f"📊 Total questions saved: {len(rows)}")
    
    return filename

# Save your current MCQs to CSV
if quiz_data:
    csv_filename = save_mcqs_to_csv(quiz_data)
    
    # Display preview of the CSV
    print(f"\n�� CSV Preview:")
    print("=" * 80)
    df_preview = pd.read_csv(csv_filename)
    print(df_preview.head())
    
    print(f"\n📁 File saved as: {csv_filename}")
else:
    print("No quiz data available to save")

✅ MCQs saved to CSV: computer_vision_mcqs_20250703_052155.csv
📊 Total questions saved: 5

�� CSV Preview:
   Question_Number                                           Question  \
0                1       What is the primary goal of computer vision?   
1                2  What type of data can be used in computer vision?   
2                3  What is a subdiscipline of computer vision tha...   
3                4  What is the term for the process of transformi...   
4                5  What is the goal of computer vision in the con...   

                                    Option_A  \
0  To create artificial intelligence systems   
1                             Only 2D images   
2                       Scene reconstruction   
3                           Image processing   
4              To create autonomous vehicles   

                                Option_B  \
0     To extract information from images   
1                   Only 3D point clouds   
2                       Object det

In [34]:
review = response.get("review")
print(review)

**COMPLEXITY ANALYSIS**

The quiz appears to be well-structured and covers a range of topics in computer vision. However, some questions may be too easy for advanced students, while others may be too complex for beginners. Overall, the quiz is suitable for intermediate-level students who have a basic understanding of computer vision concepts.

**QUALITY ASSESSMENT**

The quiz questions are clear and concise, with accurate grammar and punctuation. The explanations provided for each question are helpful and educational. However, some questions may benefit from additional context or examples to enhance understanding.

**ENHANCED QUIZ**

Here is the updated quiz with improved questions, tone, and difficulty level:

```
{
  "quiz_info": {
    "title": "Computer Vision Quiz",
    "subject": "Computer Vision",
    "difficulty": "intermediate",
    "total_questions": 5
  },
  "questions": [
    {
      "id": 1,
      "question": "What is the primary goal of computer vision in image processing?